# ViT model For image Classification


In [1]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from tqdm import tqdm
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

In [2]:
TRAIN = r'C:\Users\roysu\Desktop\Project\test_folder\val'
VAL = r'C:\Users\roysu\Desktop\Project\test_folder\train'

# TRAIN = "/home/udai/Desktop/Data/train_1"
# VAL = "/home/udai/Desktop/Data/test_1"

INPUT_HEIGHT = 256
INPUT_WIDTH = 256

BATCH_SIZE = 8
VAL_SPLIT = 0.1
EPOCH=25

In [3]:
resize = transforms.Resize(size=(INPUT_HEIGHT,INPUT_WIDTH))
hFlip = transforms.RandomHorizontalFlip(p=0.25)
vFlip = transforms.RandomVerticalFlip(p=0.25)
rotate = transforms.RandomRotation(degrees=15)

trainTransforms = transforms.Compose([resize, hFlip, vFlip, rotate,transforms.ToTensor()])
valTransforms = transforms.Compose([resize, transforms.ToTensor()])

trainDataset = ImageFolder(root=TRAIN,transform=trainTransforms,)
valDataset = ImageFolder(root=VAL, transform=valTransforms)

print("Training dataset contains {} samples...".format(len(trainDataset)))
print("Validation dataset contains {} samples...".format(len(valDataset)))

trainloader = DataLoader(trainDataset,batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(valDataset, batch_size=BATCH_SIZE)


Training dataset contains 19033 samples...
Validation dataset contains 76601 samples...


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(device)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler(enabled=False)

cpu


In [14]:
def train(net, epoch):
    print('\nEpoch: %d' % epoch)
    optimizer = optim.Adam(net.parameters(), lr=1e-4)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    print("Start Training")
    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader)):
        inputs, targets = inputs.to(device), targets.to(device)
        # Train with amp
        with torch.cuda.amp.autocast(enabled=False):
            outputs = net(inputs)
            loss = criterion(outputs, targets)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

#         if batch_idx == len(trainloader) - 1:
#               print('train', batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                 % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    return round(train_loss/(batch_idx+1),2),round(100.*correct/total,2)


def test(net, epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(testloader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

#             if batch_idx == len(testloader) - 1:
#                   print('test', batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
#                     % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    acc = round(100.*correct/total,2)

    return round(test_loss,2), acc

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

In [15]:
class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
          nn.Linear(dim, hidden_dim),
          nn.GELU(),
          nn.Dropout(dropout),
          nn.Linear(hidden_dim, dim),
          nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [16]:
class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64, dropout = 0.):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.attend = nn.Softmax(dim = -1)
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

        self.to_out = nn.Sequential(
          nn.Linear(inner_dim, dim),
          nn.Dropout(dropout)
          ) if project_out else nn.Identity()
    def forward(self, x):
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

In [17]:
class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Attention(dim, heads = heads, dim_head = dim_head, dropout = dropout)),
                PreNorm(dim, FeedForward(dim, mlp_dim, dropout = dropout))
                ]))
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
            return x

In [18]:
class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (cls token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
          Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width), # from 512,3,32,32 -> 512, 64, 48
          nn.Linear(patch_dim, dim),
          )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

In [19]:
net = ViT(
    image_size = INPUT_HEIGHT,
    patch_size = 16,
    num_classes = 2,
    dim = BATCH_SIZE,
    depth = 6,
    heads = 8,
    mlp_dim = 512,
    dropout = 0.1,
    emb_dropout = 0.1
)

In [20]:
from torchsummary import summary
summary(net,(3,256,256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Rearrange-1             [-1, 256, 768]               0
            Linear-2               [-1, 256, 8]           6,152
           Dropout-3               [-1, 257, 8]               0
         LayerNorm-4               [-1, 257, 8]              16
            Linear-5            [-1, 257, 1536]          12,288
           Softmax-6          [-1, 8, 257, 257]               0
            Linear-7               [-1, 257, 8]           4,104
           Dropout-8               [-1, 257, 8]               0
         Attention-9               [-1, 257, 8]               0
          PreNorm-10               [-1, 257, 8]               0
        LayerNorm-11               [-1, 257, 8]              16
           Linear-12             [-1, 257, 512]           4,608
             GELU-13             [-1, 257, 512]               0
          Dropout-14             [-1, 2

In [21]:
net = torch.nn.DataParallel(net)

In [63]:
import datetime
import os
time=datetime.datetime.now()
output_folder="Output_at_{}-{}-{}_H{}M{}".format(time.day,time.month,time.year,time.hour,time.minute)

os.mkdir(output_folder)
cwd=os.getcwd()


for i in range(EPOCH):
    t_loss,t_acc=train(net, i)
    v_loss,v_acc=test(net, i)
    print(f"Epoch : {i} Train_loss : {t_loss} Train_acc : {t_acc} Test_loss : {v_loss} Test_acc {v_acc} ")
    with open(file=os.path.join(cwd,output_folder,"model_logs.txt"), mode="a") as log:
        log.write(f"Epoch : {i} Train_loss : {t_loss} Train_acc : {t_acc} Test_loss : {v_loss} Test_acc {v_acc} \n")
    torch.save(net,os.path.join(cwd,output_folder,f"model_at_epoch_{i}.model"))


Epoch: 0
Start Training


  0%|                                                                                         | 0/9576 [00:00<?, ?it/s]


Epoch : 0 Train_loss : 0.38 Train_acc : 87.5 Test_loss : 0.16 Test_acc 100.0 

Epoch: 1
Start Training


  0%|                                                                                         | 0/9576 [00:00<?, ?it/s]


Epoch : 1 Train_loss : 0.38 Train_acc : 87.5 Test_loss : 0.16 Test_acc 100.0 

Epoch: 2
Start Training


...

Epoch : 22 Train_loss : 0.38 Train_acc : 87.5 Test_loss : 0.14 Test_acc 100.0 

Epoch: 23
Start Training


  0%|                                                                                         | 0/9576 [00:00<?, ?it/s]


Epoch : 23 Train_loss : 1.02 Train_acc : 50.0 Test_loss : 0.15 Test_acc 100.0 

Epoch: 24
Start Training


  0%|                                                                                         | 0/9576 [00:00<?, ?it/s]

Epoch : 24 Train_loss : 0.4 Train_acc : 87.5 Test_loss : 0.14 Test_acc 100.0 


In [61]:
time.hour

11